## Convolutional Neural Networks

En esta actividad, vamos a trabajar con Convolutional Neural Networks para resolver un problema de clasificación de imágenes. En particular, vamos a clasificar imágenes de personajes de la conocida serie de los Simpsons.

Como las CNN profundas son un tipo de modelo bastante avanzado y computacionalmente costoso, se recomienda hacer la práctica en Google Colaboratory con soporte para GPUs. En [este enlace](https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d) se explica cómo activar un entorno con GPUs. *Nota: para leer las imágenes y estandarizarlas al mismo tamaño se usa la librería opencv. Esta ĺibrería está ya instalada en el entorno de Colab, pero si trabajáis de manera local tendréis que instalarla.*

<center><img src="https://i.imgur.com/i8zIGqX.jpg" style="text-align: center" height="300px"></center>

El dataset a utilizar consiste en imágenes de personajes de los Simpsons extraídas directamente de capítulos de la serie. Este dataset ha sido recopilado por [Alexandre Attia](https://medium.com/@alexattia18) y es más complejo que el dataset de Fashion MNIST que hemos utilizado hasta ahora. Aparte de tener más clases (vamos a utilizar los 18 personajes con más imágenes), los personajes pueden aparecer en distintas poses, en distintas posiciones de la imagen o con otros personajes en pantalla (si bien el personaje a clasificar siempre aparece en la posición predominante).

El dataset de training puede ser descargado desde aquí:

[Training data](https://drive.google.com/drive/folders/171GKxRx2SCjLfcfYSPbIa1HzpR6ha3-s?usp=sharing) (~500MB)

Por otro lado, el dataset de test puede ser descargado de aquí:

[Test data](https://drive.google.com/drive/folders/1B4L6Pk1x9cbfvbP1uQTBSD959_n3QdvL?usp=sharing) (~10MB)

Antes de empezar la práctica, se recomienda descargar las imágenes y echarlas un vistazo.

Todo el dataset en un solo enlace([Enlace_descarga_completo](https://drive.google.com/file/d/1DeToSr-V_BJn3FTRD40J2HvlB8BNP_La/view?usp=sharing))


## Carga de los datos

In [1]:
from google.colab.patches import cv2_imshow
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization, MaxPooling2D
from keras.optimizers import Adam, SGD
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import itertools
import glob
%matplotlib inline

# Crear directorios base
!mkdir -p /content/simpsons

# Descargar el archivo del dataset
!gdown 1DeToSr-V_BJn3FTRD40J2HvlB8BNP_La -O /content/archive.zip

# Descomprimir el archivo ZIP
!unzip -q /content/archive.zip -d /content/simpsons

# Verificar la estructura extraída
print("Estructura de carpetas extraídas:")
!ls -la /content/simpsons

# Según las imágenes proporcionadas, configuramos las rutas correctas
# Carpetas de entrenamiento y prueba según tu estructura
DATASET_TRAIN_PATH = "/content/simpsons/simpsons_dataset"
DATASET_TEST_PATH = "/content/simpsons/kaggle_simpson_testset"

# Verificar que las carpetas existen
print(f"\nCarpeta de entrenamiento existe: {os.path.exists(DATASET_TRAIN_PATH)}")
print(f"Carpeta de prueba existe: {os.path.exists(DATASET_TEST_PATH)}")

# Contar personajes y mostrar estadísticas si las carpetas existen
if os.path.exists(DATASET_TRAIN_PATH):
    # Comprobar si hay subcarpetas (personajes)
    if any(os.path.isdir(os.path.join(DATASET_TRAIN_PATH, f)) for f in os.listdir(DATASET_TRAIN_PATH)):
        train_classes = [d for d in os.listdir(DATASET_TRAIN_PATH) if os.path.isdir(os.path.join(DATASET_TRAIN_PATH, d))]
        print(f"\nNúmero de personajes en el dataset de entrenamiento: {len(train_classes)}")
        print(f"Ejemplos de personajes: {train_classes[:5]}")

        # Contar imágenes por personaje
        train_images_count = {}
        total_train_images = 0

        for character in train_classes:
            char_path = os.path.join(DATASET_TRAIN_PATH, character)
            images = [f for f in os.listdir(char_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            train_images_count[character] = len(images)
            total_train_images += len(images)

        print(f"Total de imágenes de entrenamiento: {total_train_images}")
        print("\nPersonajes con más imágenes:")
        for char, count in sorted(train_images_count.items(), key=lambda x: x[1], reverse=True)[:5]:
            print(f"{char}: {count} imágenes")
    else:
        print("La carpeta de entrenamiento no parece contener subdirectorios de personajes.")
        print("Contenido de la carpeta de entrenamiento:")
        !ls -la {DATASET_TRAIN_PATH}

if os.path.exists(DATASET_TEST_PATH):
    # Comprobar si hay subcarpetas (personajes) o si es una estructura plana
    if any(os.path.isdir(os.path.join(DATASET_TEST_PATH, f)) for f in os.listdir(DATASET_TEST_PATH)):
        test_classes = [d for d in os.listdir(DATASET_TEST_PATH) if os.path.isdir(os.path.join(DATASET_TEST_PATH, d))]
        print(f"\nNúmero de personajes en el dataset de prueba: {len(test_classes)}")
        print(f"Ejemplos de personajes: {test_classes[:5]}")

        # Contar imágenes totales en prueba
        test_images = 0
        for cls in test_classes:
            test_images += len(os.listdir(os.path.join(DATASET_TEST_PATH, cls)))
        print(f"Total de imágenes de prueba: {test_images}")
    else:
        # Si no hay subcarpetas, contar directamente las imágenes
        test_images = [f for f in os.listdir(DATASET_TEST_PATH) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        print(f"\nTotal de imágenes de prueba: {len(test_images)}")

# Ahora puedes continuar con tu código para el modelo de clasificación
print("\nConfiguración completa. Puedes proceder con el entrenamiento del modelo.")

Downloading...
From (original): https://drive.google.com/uc?id=1DeToSr-V_BJn3FTRD40J2HvlB8BNP_La
From (redirected): https://drive.google.com/uc?id=1DeToSr-V_BJn3FTRD40J2HvlB8BNP_La&confirm=t&uuid=cd156c91-7103-4357-bbf6-bdbc13872199
To: /content/archive.zip
100% 1.16G/1.16G [00:16<00:00, 68.5MB/s]
Estructura de carpetas extraídas:
total 81304
drwxr-xr-x  4 root root     4096 May  2 15:22 .
drwxr-xr-x  1 root root     4096 May  2 15:22 ..
-rw-r--r--  1 root root   491788 Sep 20  2019 annotation.txt
-rw-r--r--  1 root root   598494 Sep 20  2019 characters_illustration.png
drwxr-xr-x  3 root root     4096 May  2 15:22 kaggle_simpson_testset
-rw-r--r--  1 root root     1452 Sep 20  2019 number_pic_char.csv
drwxr-xr-x 45 root root     4096 May  2 15:22 simpsons_dataset
-rw-r--r--  1 root root 82136544 Sep 20  2019 weights.best.hdf5

Carpeta de entrenamiento existe: True
Carpeta de prueba existe: True

Número de personajes en el dataset de entrenamiento: 43
Ejemplos de personajes: ['krusty_t

In [2]:
# Esta variable contiene un mapeo de número de clase a personaje.
# Utilizamos sólo los 18 personajes del dataset que tienen más imágenes.
MAP_CHARACTERS = {
    0: 'abraham_grampa_simpson', 1: 'apu_nahasapeemapetilon', 2: 'bart_simpson',
    3: 'charles_montgomery_burns', 4: 'chief_wiggum', 5: 'comic_book_guy', 6: 'edna_krabappel',
    7: 'homer_simpson', 8: 'kent_brockman', 9: 'krusty_the_clown', 10: 'lisa_simpson',
    11: 'marge_simpson', 12: 'milhouse_van_houten', 13: 'moe_szyslak',
    14: 'ned_flanders', 15: 'nelson_muntz', 16: 'principal_skinner', 17: 'sideshow_bob'
}

# Vamos a standarizar todas las imágenes a tamaño 64x64
IMG_SIZE = 64

In [3]:
def load_train_set(dirname, map_characters, verbose=True):
    """Esta función carga los datos de training en imágenes.

    Como las imágenes tienen tamaños distintas, utilizamos la librería opencv
    para hacer un resize y adaptarlas todas a tamaño IMG_SIZE x IMG_SIZE.

    Args:
        dirname: directorio completo del que leer los datos
        map_characters: variable de mapeo entre labels y personajes
        verbose: si es True, muestra información de las imágenes cargadas

    Returns:
        X, y: X es un array con todas las imágenes cargadas con tamaño
                IMG_SIZE x IMG_SIZE
              y es un array con las labels de correspondientes a cada imagen
    """
    X_train = []
    y_train = []
    for label, character in map_characters.items():
        files = os.listdir(os.path.join(dirname, character))
        images = [file for file in files if file.endswith("jpg")]
        if verbose:
          print("Leyendo {} imágenes encontradas de {}".format(len(images), character))
        for image_name in images:
            image = cv2.imread(os.path.join(dirname, character, image_name))
            X_train.append(cv2.resize(image,(IMG_SIZE, IMG_SIZE)))
            y_train.append(label)
    return np.array(X_train), np.array(y_train)

In [4]:
def load_test_set(dirname, map_characters, verbose=True):
    """Esta función funciona de manera equivalente a la función load_train_set
    pero cargando los datos de test."""
    X_test = []
    y_test = []
    reverse_dict = {v: k for k, v in map_characters.items()}
    for filename in glob.glob(dirname + '/*.*'):
        char_name = "_".join(filename.split('/')[-1].split('_')[:-1])
        if char_name in reverse_dict:
            image = cv2.imread(filename)
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            X_test.append(image)
            y_test.append(reverse_dict[char_name])
    if verbose:
        print("Leídas {} imágenes de test".format(len(X_test)))
    return np.array(X_test), np.array(y_test)


In [7]:
# Cargamos los datos. Si no estás trabajando en colab, cambia los paths por
# los de los ficheros donde hayas descargado los datos.
DATASET_TRAIN_PATH_COLAB = "/content/simpsons/simpsons_dataset"
DATASET_TEST_PATH_COLAB = "/content/simpsons/kaggle_simpson_testset/kaggle_simpson_testset"

X, y = load_train_set(DATASET_TRAIN_PATH_COLAB, MAP_CHARACTERS)
X_t, y_t = load_test_set(DATASET_TEST_PATH_COLAB, MAP_CHARACTERS)

Leyendo 913 imágenes encontradas de abraham_grampa_simpson
Leyendo 623 imágenes encontradas de apu_nahasapeemapetilon
Leyendo 1342 imágenes encontradas de bart_simpson
Leyendo 1193 imágenes encontradas de charles_montgomery_burns
Leyendo 986 imágenes encontradas de chief_wiggum
Leyendo 469 imágenes encontradas de comic_book_guy
Leyendo 457 imágenes encontradas de edna_krabappel
Leyendo 2246 imágenes encontradas de homer_simpson
Leyendo 498 imágenes encontradas de kent_brockman
Leyendo 1206 imágenes encontradas de krusty_the_clown
Leyendo 1354 imágenes encontradas de lisa_simpson
Leyendo 1291 imágenes encontradas de marge_simpson
Leyendo 1079 imágenes encontradas de milhouse_van_houten
Leyendo 1452 imágenes encontradas de moe_szyslak
Leyendo 1454 imágenes encontradas de ned_flanders
Leyendo 358 imágenes encontradas de nelson_muntz
Leyendo 1194 imágenes encontradas de principal_skinner
Leyendo 877 imágenes encontradas de sideshow_bob
Leídas 890 imágenes de test


In [8]:
# Vamos a barajar aleatoriamente los datos. Esto es importante ya que si no
# lo hacemos y, por ejemplo, cogemos el 20% de los datos finales como validation
# set, estaremos utilizando solo un pequeño número de personajes, ya que
# las imágenes se leen secuencialmente personaje a personaje.
perm = np.random.permutation(len(X))
X, y = X[perm], y[perm]

## Entregable

Utilizando Convolutional Neural Networks con Keras, entrenar un clasificador que sea capaz de reconocer personajes en imágenes de los Simpsons con una accuracy en el dataset de test de **85%**. Redactar un informe analizando varias de las alternativas probadas y los resultados obtenidos.

A continuación se detallan una serie de aspectos orientativos que podrían ser analizados en vuestro informe (no es necesario tratar todos ellos ni mucho menos, esto son ideas orientativas de aspectos que podéis explorar):

*   Análisis de los datos a utilizar.
*   Análisis de resultados, obtención de métricas de *precision* y *recall* por clase y análisis de qué clases obtienen mejores o peores resultados.
*   Análisis visual de los errores de la red. ¿Qué tipo de imágenes o qué personajes dan más problemas a nuestro modelo?
*   Comparación de modelos CNNs con un modelo de Fully Connected para este problema.
*   Utilización de distintas arquitecturas CNNs, comentando aspectos como su profundidad, hiperparámetros utilizados, optimizador, uso de técnicas de regularización, *batch normalization*, etc.
*   [ *algo más difícil* ] Utilización de *data augmentation*. Esto puede conseguirse con la clase [ImageDataGenerator](https://keras.io/preprocessing/image/#imagedatagenerator-class) de Keras.

Notas:
* Recuerda partir los datos en training/validation para tener una buena estimación de los valores que nuestro modelo tendrá en los datos de test, así como comprobar que no estamos cayendo en overfitting. Una posible partición puede ser 80 / 20.
* No es necesario mostrar en el notebook las trazas de entrenamiento de todos los modelos entrenados, si bien una buena idea seria guardar gráficas de esos entrenamientos para el análisis. Sin embargo, **se debe mostrar el entrenamiento completo del mejor modelo obtenido y la evaluación de los datos de test con este modelo**.
* Las imágenes **no están normalizadas**. Hay que normalizarlas como hemos hecho en trabajos anteriores.
* El test set del problema tiene imágenes un poco más "fáciles", por lo que es posible encontrarse con métricas en el test set bastante mejores que en el training set.